In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
import time
tqdm.tqdm.pandas()

In [2]:
%%time
concat_df = pd.read_excel('./공공주택찾기/SH_LH_거래사례_20180101_20220221.xlsx')
print(concat_df.shape)
concat_df.head()

(18547, 10)
Wall time: 2.24 s


,지번주소,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,소유자
0,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.50,25850,4,2021,2021-06-22,1202.325581,연립다세대,SH
1,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.92,26050,2,2021,2021-06-22,1188.412409,연립다세대,SH
2,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.92,25950,3,2021,2021-06-22,1183.850365,연립다세대,SH
3,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),24.42,28150,5,2021,2021-06-22,1152.743653,연립다세대,SH
4,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),25.01,29450,2,2021,2021-06-22,1177.528988,연립다세대,SH


In [3]:
concat_df['구'] = concat_df['지번주소'].apply(lambda x: x.split(' ')[1])
concat_df['동'] = concat_df['지번주소'].apply(lambda x: x.split(' ')[2])

In [4]:
concat_df.head()

,지번주소,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,소유자,구,동
0,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.50,25850,4,2021,2021-06-22,1202.325581,연립다세대,SH,강남구,개포동
1,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.92,26050,2,2021,2021-06-22,1188.412409,연립다세대,SH,강남구,개포동
2,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.92,25950,3,2021,2021-06-22,1183.850365,연립다세대,SH,강남구,개포동
3,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),24.42,28150,5,2021,2021-06-22,1152.743653,연립다세대,SH,강남구,개포동
4,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),25.01,29450,2,2021,2021-06-22,1177.528988,연립다세대,SH,강남구,개포동


In [5]:
def modify_dong(x):
    if pd.isna(x) == True:
        return x
    else:
        for i in range(10):
            x = x.replace(str(i), '')
        
        return x

In [6]:
concat_df['동'] = concat_df['동'].apply(modify_dong)

In [7]:
concat_df['구_동'] = concat_df['구'] + '_' + concat_df['동']

In [8]:
concat_df['구_동'].value_counts()

금천구_시흥동       1242
강동구_천호동        927
동대문구_장안동       744
금천구_독산동        743
강북구_수유동        572
강동구_성내동        537
구로구_개봉동        502
구로구_궁동         491
서초구_양재동        424
구로구_고척동        422
광진구_중곡동        371
노원구_상계동        363
구로구_오류동        356
성북구_장위동        346
도봉구_쌍문동        342
양천구_목동         321
송파구_오금동        306
송파구_송파동        263
금천구_가산동        246
송파구_석촌동        242
강동구_암사동        233
은평구_역촌동        219
은평구_구산동        215
강서구_화곡동        211
송파구_방이동        201
송파구_가락동        195
서초구_서초동        192
동작구_사당동        189
중랑구_상봉동        186
영등포구_대림동       185
양천구_신정동        180
은평구_신사동        179
서초구_방배동        174
강동구_길동         173
양천구_신월동        172
성북구_석관동        171
강남구_역삼동        170
도봉구_방학동        169
중랑구_중화동        167
노원구_공릉동        166
광진구_자양동        165
송파구_삼전동        162
강북구_우이동        161
중랑구_면목동        154
성북구_하월곡동       150
서대문구_홍제동       149
동대문구_이문동       149
도봉구_창동         140
은평구_녹번동        131
성북구_정릉동        128
종로구_연지동        128
동작구_상도동        120
중랑구_묵동      

In [9]:
concat_df['구_동'].nunique()

122

In [10]:
concat_df['구'].nunique()

24

In [11]:
def create_price_range_statistics_df(df, tradetype='매매', real_estate_type='연립다세대'):
    range_statistics_df = pd.DataFrame()
    range_statistics_df['전용면적구간'] = np.nan
    range_statistics_df['거래대상수'] = np.nan
    range_statistics_df['평균거래단가(만원/㎡)'] = np.nan
    df = df.copy()
    
    for i, v in enumerate(range(17, 40)):
        range_min = v
        range_max = v + 1

        range_statistics_df.loc[i, '전용면적구간'] = '{}이상 {}미만'.format(range_min, range_max)

        range_df = df[
            (df['전용면적(㎡)'] >= range_min)
            &(df['전용면적(㎡)'] < range_max)
            #&(df['거래구분'] == tradetype)
            &(df['부동산유형'] == real_estate_type)
        ]

        range_statistics_df.loc[i, '거래대상수'] = range_df.shape[0]
        range_statistics_df.loc[i, '평균거래단가(만원/㎡)'] = range_df['단가(만원/㎡)'].mean()
        
    for i, v in enumerate(range(40, 65, 5)):
        range_min = v
        range_max = v + 5

        i = i+23

        range_statistics_df.loc[i, '전용면적구간'] = '{}이상 {}미만'.format(range_min, range_max)

        range_df = df[
            (df['전용면적(㎡)'] >= range_min)
            &(df['전용면적(㎡)'] < range_max)
            #&(df['거래구분'] == tradetype)
            &(df['부동산유형'] == real_estate_type)
        ]

        range_statistics_df.loc[i, '거래대상수'] = range_df.shape[0]
        range_statistics_df.loc[i, '평균거래단가(만원/㎡)'] = range_df['단가(만원/㎡)'].mean()
        
    range_statistics_df.loc[28, '전용면적구간'] = '{}이상 {}미만'.format(65, 100)

    range_df = df[
        (df['전용면적(㎡)'] >= 65)
        &(df['전용면적(㎡)'] < 100)
        #&(df['거래구분'] == tradetype)
        &(df['부동산유형'] == real_estate_type)
    ]

    range_statistics_df.loc[28, '거래대상수'] = range_df.shape[0]
    range_statistics_df.loc[28, '평균거래단가(만원/㎡)'] = range_df['단가(만원/㎡)'].mean()
    
    return range_statistics_df

In [12]:
basedir = './공공주택찾기/'

In [13]:
concat_df.head()

,지번주소,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,소유자,구,동,구_동
0,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.50,25850,4,2021,2021-06-22,1202.325581,연립다세대,SH,강남구,개포동,강남구_개포동
1,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.92,26050,2,2021,2021-06-22,1188.412409,연립다세대,SH,강남구,개포동,강남구_개포동
2,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.92,25950,3,2021,2021-06-22,1183.850365,연립다세대,SH,강남구,개포동,강남구_개포동
3,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),24.42,28150,5,2021,2021-06-22,1152.743653,연립다세대,SH,강남구,개포동,강남구_개포동
4,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),25.01,29450,2,2021,2021-06-22,1177.528988,연립다세대,SH,강남구,개포동,강남구_개포동


In [14]:
try:
    os.makedirs('./공공주택찾기/서울시전체_통계자료/')
except:
    pass

In [15]:
try:
    os.makedirs('./공공주택찾기/서울시전체_통계자료/전체/')
except:
    pass

try:
    os.makedirs('./공공주택찾기/서울시전체_통계자료/최근1년/')
except:
    pass

In [16]:
concat_df['부동산유형'].value_counts()

연립다세대    16516
오피스텔      1602
아파트        429
Name: 부동산유형, dtype: int64

In [17]:
data_ranges = ['전체', '최근1년']

In [18]:
date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')
date_today

Timestamp('2022-02-22 00:00:00')

In [19]:
def visualise(df, range_statistics_df, basedir, re_type, data_range, loc):
    df = df.copy()
    range_statistics_df = range_statistics_df.copy()
    
    font = {'size': 16}
    matplotlib.rc('font', **font)
    f, ax = plt.subplots(figsize=(20,10))
    #sns.factorplot(x='전용면적구간', y='평균거래단가(만원/㎡)', hue='거래유형', data=concat_df, kind='bar', size=15, aspect=4/3)
    sns.barplot(x='전용면적구간', y='평균거래단가(만원/㎡)', data=range_statistics_df)
    plt.xticks(rotation=40)
    plt.title('{} {} {} 전용면적구간별 평균거래단가'.format(loc, data_range, re_type))
    plt.tight_layout()
    plt.savefig(basedir + '{} {} {} 전용면적구간별_평균거래단가'.format(loc, data_range, re_type))
    plt.clf()
    
    font = {'size': 16}
    matplotlib.rc('font', **font)
    f, ax = plt.subplots(figsize=(40, 20))
    if df.shape[0] > 0:
        ax.set_ylim([df['단가(만원/㎡)'].min() - 200, df['단가(만원/㎡)'].max() + 200])
    plt.title('{} {} {} {}건의 전용면적 단가와 계약날짜'.format(loc, data_range, re_type, df.shape[0]))
    plt.grid(True)
    ax.xaxis.update_units(df['계약날짜'])
    sns.regplot(x=ax.xaxis.convert_units(df['계약날짜']), y=df['단가(만원/㎡)'])
    plt.xticks(rotation=40)
    if df['계약날짜'].nunique() > 30:
        ax.xaxis.set_major_locator(plt.MaxNLocator(30))
    plt.savefig(basedir + '{} {} {} {}건의 전용면적 단가와 계약날짜'.format(loc, data_range, re_type, df.shape[0]))
    plt.clf()
    
    font = {'size': 12}
    matplotlib.rc('font', **font)
    f, ax = plt.subplots(figsize=(12,6))
    plt.title('{} {} {} {}건의 전용면적 단가 분포'.format(loc, data_range, re_type, df.shape[0]))
    plt.grid(True)
    sns.histplot(re_type_df['단가(만원/㎡)'], kde=True)
    plt.savefig(basedir + '{} {} {} {}건의 전용면적 단가 분포'.format(loc, data_range, re_type, df.shape[0]))
    plt.clf()

In [20]:
date_today

Timestamp('2022-02-22 00:00:00')

In [21]:
type(date_today)

pandas._libs.tslibs.timestamps.Timestamp

In [22]:
for data_range in data_ranges:
    for re_type in concat_df['부동산유형'].unique():

        basedir = './공공주택찾기/서울시전체_통계자료/' + data_range + '/'
        try:
            os.makedirs(basedir)
        except:
            pass

        re_type_df = concat_df[concat_df['부동산유형'] == re_type].sort_values(['계약날짜']).reset_index(drop=True)
        re_type_df['계약날짜'] = pd.to_datetime(re_type_df['계약날짜'])
        if data_range == '최근1년':
            re_type_df = re_type_df[re_type_df['계약날짜'] >= (date_today - pd.DateOffset(months=12))]

        range_statistics_df = create_price_range_statistics_df(re_type_df, real_estate_type=re_type)
        range_statistics_df.to_excel(basedir + '{}_전용면적구간별_통계자료_베이스.xlsx'.format(re_type), index=False)
        
        visualise(re_type_df, range_statistics_df, basedir, re_type, data_range, '서울시 전체')

<Figure size 1440x720 with 0 Axes>

<Figure size 2880x1440 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 2880x1440 with 0 Axes>

...

<Figure size 2880x1440 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 2880x1440 with 0 Axes>

<Figure size 864x432 with 0 Axes>

In [23]:
try:
    os.makedirs('./공공주택찾기/구별_자료/')
except:
    pass

In [24]:
concat_df.head()

,지번주소,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,소유자,구,동,구_동
0,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.50,25850,4,2021,2021-06-22,1202.325581,연립다세대,SH,강남구,개포동,강남구_개포동
1,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.92,26050,2,2021,2021-06-22,1188.412409,연립다세대,SH,강남구,개포동,강남구_개포동
2,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),21.92,25950,3,2021,2021-06-22,1183.850365,연립다세대,SH,강남구,개포동,강남구_개포동
3,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),24.42,28150,5,2021,2021-06-22,1152.743653,연립다세대,SH,강남구,개포동,강남구_개포동
4,서울특별시 강남구 개포동 1166-9,동영빌(1166-9),25.01,29450,2,2021,2021-06-22,1177.528988,연립다세대,SH,강남구,개포동,강남구_개포동


In [25]:
concat_df['구'] = concat_df['지번주소'].apply(lambda x: x.split(' ')[1])
concat_df['동'] = concat_df['지번주소'].apply(lambda x: x.split(' ')[2])

In [26]:
concat_df.isna().sum()

지번주소        0
건물(단지)명     0
전용면적(㎡)     0
거래금액(만원)    0
층           0
건축년도        0
계약날짜        0
단가(만원/㎡)    0
부동산유형       0
소유자         0
구           0
동           0
구_동         0
dtype: int64

In [27]:
concat_df['단가(만원/㎡)'].describe()

count    18547.000000
mean       796.795125
std        259.012706
min        272.238481
25%        602.530629
50%        716.495842
75%        934.015927
max       1951.666056
Name: 단가(만원/㎡), dtype: float64

In [28]:
for gu in tqdm.tqdm_notebook(concat_df['구'].unique()):
    gu_df = concat_df[concat_df['구'] == gu]
    for data_range in data_ranges:
        for re_type in gu_df['부동산유형'].unique():

            basedir = './공공주택찾기/구별_자료/' + gu + '/' + data_range + '/'
            try:
                os.makedirs(basedir)
            except:
                pass

            re_type_df = gu_df[gu_df['부동산유형'] == re_type].sort_values(['계약날짜']).reset_index(drop=True)
            re_type_df['계약날짜'] = pd.to_datetime(re_type_df['계약날짜'])
            if data_range == '최근1년':
                re_type_df = re_type_df[re_type_df['계약날짜'] >= (date_today - pd.DateOffset(months=12))]

            range_statistics_df = create_price_range_statistics_df(re_type_df, real_estate_type=re_type)
            range_statistics_df.to_excel(basedir + '{}_전용면적구간별_통계자료_베이스.xlsx'.format(re_type), index=False)

            visualise(re_type_df, range_statistics_df, basedir, re_type, data_range, gu)

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/24 [00:00<?, ?it/s]

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


<Figure size 1440x720 with 0 Axes>

<Figure size 2880x1440 with 0 Axes>

...

<Figure size 2880x1440 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 2880x1440 with 0 Axes>

<Figure size 864x432 with 0 Axes>

In [29]:
concat_df['구 동'] = concat_df['구'] + ' ' + concat_df['동']

In [30]:
for gu in tqdm.tqdm_notebook(concat_df['구 동'].unique()):
    gu_df = concat_df[concat_df['구 동'] == gu]
    for data_range in data_ranges:
        for re_type in gu_df['부동산유형'].unique():

            basedir = './공공주택찾기/동별_자료/' + gu + '/' + data_range + '/'
            try:
                os.makedirs(basedir)
            except:
                pass

            re_type_df = gu_df[gu_df['부동산유형'] == re_type].sort_values(['계약날짜']).reset_index(drop=True)
            re_type_df['계약날짜'] = pd.to_datetime(re_type_df['계약날짜'])
            if data_range == '최근1년':
                re_type_df = re_type_df[re_type_df['계약날짜'] >= (date_today - pd.DateOffset(months=12))]

            range_statistics_df = create_price_range_statistics_df(re_type_df, real_estate_type=re_type)
            range_statistics_df.to_excel(basedir + '{}_전용면적구간별_통계자료_베이스.xlsx'.format(re_type), index=False)

            visualise(re_type_df, range_statistics_df, basedir, re_type, data_range, gu)

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/127 [00:00<?, ?it/s]

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


<Figure size 1440x720 with 0 Axes>

<Figure size 2880x1440 with 0 Axes>

...

<Figure size 2880x1440 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 2880x1440 with 0 Axes>

<Figure size 864x432 with 0 Axes>